# System de clusterisation de données web

### Liste des dépendance :
* sklearn
* numpy
* collections
* pickle

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity as cos_sim

from collections import defaultdict
import numpy as np
from numpy.linalg import norm
from random import shuffle

import pickle

#Debug
from IPython.display import clear_output
clear_output()

### Lecture de fichier
* Prend un chemin vers un fichier.
  * Le fichier doit être au format élément séparteur vecteur ( sous la forme nombre séparateur nombre ... )
  * Le séparateur est un espace
  * La normalisation des vecteurs n'est pas nécessaire
* Renvoie un tuple contenant la liste des éléments ainsi que leur représentation vectorielle.

In [ ]:
def read_vector_file(path_to_file):
    keys = []
    vectors = []
    with open(path_to_file) as file :
        line = file.readline()
        while line :
            data = line[:-1].split(" ")
            web, vector = data[0], [float(s) for s in data[1:]]
            vectors.append(vector)
            keys.append(web)
            line = file.readline()
    return (keys, vectors)

### Fonctions de préparation des données :
* Prend en entrée, une liste de site, une de vecteurs associés, une de labels associés
* Nettoie certaines catégories : "Régional"
* Renvoie 2 dictionnaires

In [ ]:
def do_someting(websites, vectors, classed_file, verbose = False) :
    site2ind_class = {}
    ind2site = []
    good_vecs = []
    count = defaultdict(int)
    keys = set(websites)
    with open(classed_file) as file :
            i = 0
            line = file.readline()
            while line :
                cat, site = line[:-1].split("\t")
                if site in keys :
                    cat = cat.split("/")[3]
                    if site not in site2ind_class and cat != 'Régional':
                        site2ind_class[site] = (i, cat)
                        count[cat]+=1
                        ind2site.append(site)
                        good_vecs.append(vectors[websites.index(site)])
                        i+=1
                line = file.readline()
    if verbose :
        pp.pprint(count)
    return site2ind_class, ind2site, good_vecs

### Chargement des données

In [ ]:
#### VAR ZONE ####
file_path = "fr.up.seeds.txt.shuf.10000" 
classed_file = "dmozFull.fr"
nb_cluster = 10
## END VAR ZONE ##

#### EXEC ZONE ####
websites, vectors = read_vector_file(file_path)
site2ind_class, ind2site, good_vecs = do_someting(websites, vectors, classed_file)
## END EXEC ZONE ##

### Clustering des données : approche k_means
* Prend un nombre de cluster et une liste de vecteurs
* Renvoie les clusters associés

In [ ]:
def run_k_means(vectors, nb_clus):
    kmeans = MiniBatchKMeans(n_clusters = nb_clus, 
                batch_size = 10000,
                max_iter = 200)
    classif = kmeans.fit_predict(vectors)
    return classif, kmeans.cluster_centers_ 

### Clustering des données : approche HAC
* Prend une liste de de vecteurs
* Les mélanges aléatoirement
* Initialise :
    * En prend 200
    * Calcule une matrice de similarité entre les 200
    * Effectue une fusion
        * Trouver le max
        * Fusionner
    * Recalcule la similarité
    * Se souvient de la position supprimée.
* Boucle :
    * Prend un vecteur
    * Calcul la similarité
    * Effectue une fusion
        * Trouver le max
        * Fusionner
    * Recalcule la similarité
    * Se souvient de la position supprimée.

In [ ]:
class HAC :
    
    def __init__(self, batch_size = 20, min_merge = 0.7, max_passe = 2, loss = 0):
        
        #Liste de cluster à accès rapide = dic: int->set
        self.clusters = {}
        
        #Similarité minimale pour fusionner = float
        self.min_merge = min_merge
        
        #Nombre de passes = int
        self.max_passe = max_passe
        
        #Facteur de diminution de la valeur minimale de fusion = float
        self.loss = loss
        
        #Matrice de similarité = list of list of int
        self.sim_matrix = np.asarray([np.zeros(batch_size) for j in range(batch_size)])
        
        #Liste des centroids dans le batch = list of np.array 
        self.centroids = []
        
        #Taille du batch : int
        self.batch_size = batch_size
        
        #Index du dernier élément supprimé : int
        self.last = batch_size
    
    def fusion(self, merger, to_merge) :
        # Effectue la fusion de deux clusteurs en recalculant le centroid puis en fusionnant les set des clusters.
        self.centroids[merger] = np.average([self.centroids[merger], self.centroids[to_merge]],
                                            weights = [len(self.clusters[merger]), len(self.clusters[to_merge])],
                                            axis = 0) 
        self.clusters[merger] = self.clusters[merger].union(self.clusters[to_merge])

    def regen_sim(self, new) :
        #Recalcule la matrice de similarité dans la ligne/colonne donnée en argument 
        k = np.concatenate(cos_sim(self.centroids, self.centroids[new].reshape(1, -1)))
        for i, val in enumerate(k) :
            self.sim_matrix[i][new] = val
        self.sim_matrix[new] = k
        self.sim_matrix[new][new] = -1

    def return_alone(self) :
        # Renvoie aléatoirement un cluster singleton
        sorry = []
        for ind,clus in self.clusters.items() :
            if len(clus) == 1 :
                sorry.append((ind, clus.copy().pop()))
        shuffle(sorry)
        self.last = sorry[0][0]
        return sorry[0][1]
    
    def clusterize(self) : 
        # Enchaine les processus de fusions :
        """Trouver la paire avec la plus haute similarité,
            Vérifier si la similarité est suffisement élevée,
                Effectue la fusion,
                Regénère la matrice de similarité,
                Memorise l'élément fusionné
            Sinon, suprimme un cluster singleton        
        """
        merger, to_merge = np.unravel_index(np.argmax(np.array(self.sim_matrix)), (self.batch_size, self.batch_size))
        if self.min_merge < self.sim_matrix[merger][to_merge] :
            self.fusion(merger, to_merge)
            self.regen_sim(merger)
            self.last = to_merge
            return None
        else :
            return self.return_alone()
    
    def init_hac(self, vector_batch, indexes) :
        
        # Calcule une matrice de similarité entre les 200
        for i in range(self.batch_size) :
            self.clusters[i] = {indexes[i]}
            self.centroids.append(np.asarray(vector_batch[i]))
        self.sim_matrix = cos_sim(self.centroids, self.centroids)
        for i in range(self.batch_size) :
            self.sim_matrix[i][i] = -1
        return self.clusterize()
    
    def add_vector(self, vector, index) :
        
        self.clusters[self.last] = {index}
        self.centroids[self.last] = np.asarray(vector)
        self.regen_sim(self.last)
        return self.clusterize()
    
    def looper(self, indexes, vectors) :
        fail = []
        k = 1
        for i in indexes :
            ind = self.add_vector(vectors[i],i)
            if ind :
                fail.append(ind)
            print("\r{:.2%}".format(k/len(indexes)), end = '')
            k+=1
        self.min_merge -= self.loss
        return fail
    
    def run_hac(self, vectors) :
        order = [i for i in range(len(vectors))]
        shuffle(order)
        fail = []
        ind = self.init_hac([vectors[i] for i in order[:200]],order[:200])
        if ind :
            fail.append(ind)
        fail += self.looper(order[200:], vectors)
        i = 1
        while i < self.max_passe and fail :
            print('\nEchecs, passe', i, ':', len(fail))
            shuffle(fail)
            fails = fail.copy()
            fail = self.looper(fails, vectors)
            i+=1
        print('\nEchecs restants :', len(fail))
            
        return self.clusters
    
    def labelise(clusters, data_size) :
        labels = [-i-1 for i in range(data_size)]
        for i, clus in clusters.items():
            for ind in clus :
                labels[ind] = i
        return labels

### Phase exécutable / de test 

**Usage de la metric silhouette**
The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.
The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.


In [ ]:
#### K-means ZONE ####
#labels, centroids = run_k_means(good_vecs, nb_cluster)
## END K-means ZONE ##

### TEST ZONE ##

hac = HAC(batch_size = 200, min_merge = 0.50, max_passe = 2, loss = 0.025)

%prun hac_clus = HAC.labelise(hac.run_hac(good_vecs), len(good_vecs))


In [ ]:
dic = hac_clus
f = open("clus.pkl","wb")
pickle.dump(dic,f)
f.close()